In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(
            base_url = "https://api.fanar.qa/v1",
            api_key  = os.environ["FANAR_API_KEY"],
        )
        # Option A – set a default so you don’t repeat `model=…` later
        client.default_params = {"model": "Fanar-C-1-8.7B"}
        return client    
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="fanar"
deployment = APIKeyManager(model_type, "./azure.env")


In [3]:
class TopicClassifier:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def classify_topic(self, topic, information):
        """
        Classify podcast topic and determine optimal approach
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            
        Returns:
            JSON with classification results
        """
        
        prompt = f"""
أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي طبيعي وجذاب.

المهمة: حلل الموضوع التالي وحدد أفضل نهج لتقديمه في بودكاست عربي.

الموضوع: {topic}
المعلومات الأساسية: {information}

قم بتحليل الموضوع وإرجاع النتيجة بصيغة JSON تحتوي على:

{{
    "primary_category": "الفئة الرئيسية",
    "category_justification": "سبب اختيار هذه الفئة بناءً على طبيعة الموضوع",
    "optimal_style": "الأسلوب الأمثل للمناقشة",
    "discourse_pattern": "نمط الخطاب المناسب",
    "audience_engagement_goal": "هدف تفاعل الجمهور",
    "cultural_sensitivity_level": "مستوى الحساسية الثقافية",
    "controversy_potential": "احتمالية الجدل",
    "key_discussion_angles": [
        "زوايا النقاش الرئيسية المتوقعة",
        "النقاط التي ستثير اهتمام الجمهور العربي"
    ],
    "natural_tension_points": [
        "نقاط التوتر الطبيعية في الموضوع",
        "الجوانب التي قد تثير جدلاً صحياً"
    ],
    "cultural_connection_opportunities": [
        "فرص الربط بالثقافة العربية",
        "المراجع المحلية والإقليمية ذات الصلة"
    ]
}}

الفئات المتاحة:
1. "العلوم والتكنولوجيا" - للمواضيع التقنية والعلمية والابتكارات
2. "السياسة والشؤون العامة" - للمواضيع السياسية والأحداث الجارية والقضايا العامة
3. "القضايا الاجتماعية" - للمواضيع المجتمعية والعلاقات والقيم والتحديات الاجتماعية
4. "الرياضة والترفيه" - للمواضيع الرياضية والفنية والترفيهية
5. "التاريخ والثقافة" - للمواضيع التاريخية والتراثية والثقافية

الأساليب المتاحة:
- "حواري" - حوار طبيعي وودي بين المقدم والضيف
- "تعليمي" - تركيز على الشرح والتعليم بطريقة ممتعة
- "ترفيهي" - مرح وخفيف مع لمسات فكاهية
- "تحليلي" - نقاش عميق ومتخصص وتحليلي

أنماط الخطاب:
- "رسمي" - لغة رسمية ومحترمة
- "ودي" - لغة دافئة ومألوفة
- "جدلي" - نقاش حيوي مع وجهات نظر متعددة
- "سردي" - أسلوب حكواتي وقصصي



مستوى الحساسية الثقافية:
- "عالي" - يتطلب حذراً شديداً في التعامل
- "متوسط" - يحتاج مراعاة ثقافية معتدلة  
- "منخفض" - موضوع مقبول عموماً

احتمالية الجدل:
- "عالية" - موضوع مثير للجدل بطبيعته
- "متوسطة" - قد يثير بعض الاختلافات
- "منخفضة" - موضوع مقبول عموماً

تعليمات مهمة:
- حلل الموضوع بعمق وليس بشكل سطحي
- اعتبر السياق الثقافي العربي في التحليل
- ركز على ما يجعل الموضوع جذاباً للجمهور العربي
- تأكد أن التصنيف يخدم إنتاج محتوى طبيعي وتلقائي
- لا تضع علامات ```json في البداية أو النهاية
- أرجع JSON صحيح فقط بدون أي نص إضافي
-المدة 10 دقائق هي المدة المثلى للحلقة
- "،"  لا تستخدم الفاصلة العربية 
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي احترافي. تخصصك في فهم الجمهور العربي واهتماماته."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent classification
        )
        
        return response.choices[0].message.content

   

In [4]:
# Testing Instructions:

# To test Step 1, add this to a new cell in your notebook:

# Test Step 1: Topic Classification
classifier = TopicClassifier(deployment, "Fanar-C-1-8.7B")  # Use the appropriate model for your deployment

# Test with the singlehood topic
topic = "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي"

information = '''
مع انتشار تقنيات الذكاء الاصطناعي بسرعة في العالم العربي، تزداد المخاوف حول تأثيرها على الهوية الثقافية واللغة العربية. 
تشير الدراسات إلى أن 78% من المحتوى الرقمي باللغة الإنجليزية، بينما المحتوى العربي لا يتجاوز 3%. 
معظم نماذج الذكاء الاصطناعي الحالية مدربة على بيانات غربية، مما يثير تساؤلات حول قدرتها على فهم السياق الثقافي العربي.
في المقابل، تسعى دول مثل الإمارات والسعودية لتطوير نماذج ذكاء اصطناعي عربية مثل "جايس" و"الحوراء" لمواجهة هذا التحدي.
التحدي الأكبر يكمن في كيفية الاستفادة من هذه التقنيات لتعزيز الثقافة العربية بدلاً من تهميشها، وضمان أن تخدم الذكاء الاصطناعي قيمنا ومبادئنا.
'''

# Run classification
classification_result = classifier.classify_topic(topic, information)
print("Classification Result:")
print(classification_result)



# Parse and examine the JSON
try:
    parsed_result = json.loads(classification_result)
    print(f"\nPrimary Category: {parsed_result['primary_category']}")
    print(f"Optimal Style: {parsed_result['optimal_style']}")
    print(f"Discourse Pattern: {parsed_result['discourse_pattern']}")
except:
    print("Error parsing JSON result")

Classification Result:
{
"primary_category": "القضايا الاجتماعية",
"category_justification": "يتناول الموضوع القضايا المرتبطة بالتكنولوجيا وكيف تؤثر على الهوية الثقافية واللغة العربية, وهو ضمن نطاق القضايا الاجتماعية.",
"optimal_style": "تحليلي",
"discourse_pattern": "جدلي",
"audience_engagement_goal": "إشراك المستمعين في مناقشة مستقبل استخدام الذكاء الاصطناعي في المنطقة العربية",
"cultural_sensitivity_level": "عالي",
"controversy_potential": "متوسطة",
"key_discussion_angles": [
	"تأثيرات الذكاء الاصطناعي المدرب على البيانات الغربية على اللغة العربية",
	"دور الدول العربية في تطوير الذكاء الاصطناعي محليا",
	"كيفية تحقيق توازن بين الاستفادة من الذكاء الاصطناعي والحفاظ على هويتنا الثقافية",
	"مخاطر فقدان الأصالة اللغوية والثقافية بسبب الاعتماد الزائد على الذكاء الاصطناعي الأجنبي"
],
"natural_tension_points": [
	"مقارنة فعالية الذكاء الاصطناعي الذي تم تدريبه محليا مقابل ذلك الذي تم تدريبه عالميا",
	"تقييم مدى نجاح جهود الحكومات العربية حتى الآن في مجال الذكاء الاصطناعي"
],
"cultural_connec

In [ ]:
class SimplePersonaGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_personas(self, topic, information, classification_result):
        """
        Generate simple but effective host and guest personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            
        Returns:
            JSON with simple host and guest personas
        """
        
        # Parse classification to understand the requirements
        try:
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid classification JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تصميم شخصيات البودكاست العربي.

المهمة: أنشئ مقدم وضيف بسيطين ومناسبين لهذا الموضوع.

الموضوع: {topic}
المعلومات: {information}
الفئة: {primary_category}
الأسلوب المطلوب: {optimal_style}

أرجع النتيجة بصيغة JSON بسيط:

{{
    "host": {{
        "name": "اسم المقدم",
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "guest": {{
        "name": "اسم الضيف", 
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "expertise": "مجال خبرته في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "why_good_match": "لماذا هذا المقدم والضيف مناسبان لهذا الموضوع - جملة واحدة"
}}

متطلبات:
- أسماء عربية مألوفة
- شخصيات بسيطة وقابلة للتصديق
- مناسبة للموضوع والأسلوب المطلوب
- المقدم فضولي والضيف خبير أو صاحب تجربة
- لا تضع علامات ```json
- أرجع JSON فقط
- "،"  لا تستخدم الفاصلة العربية 
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم شخصيات بودكاست بسيطة ومؤثرة. الأسلوب المطلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )
        
        return response.choices[0].message.content

   

In [8]:
persona = SimplePersonaGenerator(deployment, "Fanar-C-1-8.7B")
persona_result = persona.generate_personas(topic, information, classification_result)
print("Persona Generation Result:")
print(persona_result)
try:
    parsed_result = json.loads(persona_result)
    print(f"\nHost Name: {parsed_result['host']['name']}")
    print(f"Guest Name: {parsed_result['guest']['name']}")
except:
    print("Error parsing JSON result")

Persona Generation Result:
{
  "host": {
    "name": "لمى عبد الله",
    "age": 35,
    "background": "صحفية متخصصة في الشأن التكنولوجي في إحدى الجرائد المحلية",
    "personality": "فضولية ومهتمة بتسليط الضوء على الفوائد والتحديات",
    "speaking_style": "تطرح الأسئلة المفتوحة لتعميق النقاش وتستعرض نقاط البحث بشكل واضح"
  },
  "guest": {
    "name": "علي محسن",
    "age": 42,
    "background": "باحث في علوم الكمبيوتر وأستاذ مساعد في جامعة حكومية",
    "expertise": "ذكاء اصطناعي ومعالجة اللغة الطبيعية",
    "personality": "عقلاني ومتحمس لاستخدام الذكاء الاصطناعي لتعزيز الثقافة العربية",
    "speaking_style": "يتحدث بسهولة عن مجال تخصصه ويشارك آرائه بطريقة موضوعية"
  },
  "why_good_match": "مزيج مثالي بين اهتمام الصحفية بفهم تأثير الذكاء الاصطناعي وثروة الخبرات العلمية للأستاذ"
}

Host Name: لمى عبد الله
Guest Name: علي محسن


In [12]:
class ConversationStructureGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_conversation_structure(self, topic, information, classification_result, personas_result):
        """
        Step 3: Generate core conversation structure (V1 skeleton)
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            
        Returns:
            JSON with V1-style conversation structure (without rich dialogue content)
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided for classification or personas")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        discourse_pattern = classification.get("discourse_pattern", "")
        recommended_duration = classification.get("recommended_duration", "10 دقيقة")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
You are an expert in designing conversation structures for Arabic podcasts.

Task: Create the basic conversation structure in natural dialogue format.

Topic: {topic}
Information: {information}

Context:
- Category: {primary_category}
- Style: {optimal_style}
- Discourse Pattern: {discourse_pattern}
- Duration: {recommended_duration}

Characters:
- Host: {host_name} - {host.get('background', '')}
- Guest: {guest_name} - {guest.get('background', '')}

Create the conversation outline in JSON format:

{{
    "episode_topic": "{topic}",
    "personas": {{
        "host": {{
            "name": "{host_name}",
            "background": "{host.get('background', '')}",
            "speaking_style": "{host.get('speaking_style', '')}"
        }},
        "guest": {{
            "name": "{guest_name}",
            "background": "{guest.get('background', '')}",
            "speaking_style": "{guest.get('speaking_style', '')}"
        }}
    }},
    "conversation_flow": {{
        "intro1": {{
            "opening_line": "Actual opening line by the host",
            "podcast_introduction": "Podcast introduction related to the topic",
            "episode_hook": "Specific engaging sentence about the topic",
            "tone_guidance": "Appropriate tone for {optimal_style} style"
        }},
        "intro2": {{
            "topic_introduction": "Clear topic introduction",
            "guest_welcome": "Welcome to guest {guest_name}",
            "guest_bio_highlight": "Guest background introduction",
            "transition_to_discussion": "Transition to main discussion"
        }},
        "main_discussion": [
            {{
                "point_title": "First main point of the topic",
                "personal_angle": "How it relates to the characters' backgrounds"
            }},
            {{
                "point_title": "Second main point of the topic", 
                "personal_angle": "Personal angle for this point"
            }},
            {{
                "point_title": "Third main point of the topic",
                "personal_angle": "Personal connection and concluding angle"
            }}
        ],
        "closing": {{
            "conclusion": {{
                "main_takeaways": "Main takeaways",
                "guest_final_message": "Guest's final message",
                "host_closing_thoughts": "Host's closing thoughts"
            }},
            "outro": {{
                "guest_appreciation": "Thank the guest",
                "audience_thanks": "Thank the listeners",
                "call_to_action": "Call for engagement",
                "final_goodbye": "Final goodbye"
            }}
        }}
    }},
    "cultural_context": {{
        "proverbs_sayings": [
            "Relevant Arabic proverb for the topic",
            "Related wisdom saying"
        ],
        "regional_references": [
            "Local reference related to the topic",
            "Relevant Arab experience"
        ]
    }},
    "language_style": {{
        "formality_level": "Appropriate level for {optimal_style} style",
        "dialect_touches": "Light dialectal touches according to the host",
        "vocabulary_richness": "Vocabulary suitable for the topic"
    }},
    "technical_notes": {{
        "pacing_guidance": "Appropriate pacing for {recommended_duration} duration",
        "pause_points": "Natural pause points",
        "emphasis_moments": "Important emphasis moments"
    }}
}}

CRITICAL REQUIREMENTS:
- Write actual content, not descriptions
- Use the real character names ({host_name}, {guest_name})
- Make all content related to the topic: {topic}
- All JSON values must be in Modern Standard Arabic (MSA)
- JSON keys should be in English
- Use ONLY English commas (,) - NEVER Arabic commas (،)
- Use ONLY standard double quotes (") - NEVER Arabic quotes
- Do NOT include any explanatory text before or after the JSON
- Do NOT include ```json markers
- Return ONLY valid JSON that can be parsed by json.loads()
- Ensure every opening brace {{ has a closing brace }}
- Ensure every array [ has a closing bracket ]
- End every JSON property with a comma except the last one in each object
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in designing conversation structures for Arabic podcasts. Required style: {optimal_style}. You MUST return ONLY valid JSON with English punctuation. No explanatory text. No Arabic commas. No extra text before or after JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6  # Balanced for structure creation
        )
        
        # Clean the response to ensure valid JSON
        raw_response = response.choices[0].message.content.strip()
        
        # Remove any text before the first { and after the last }
        start_idx = raw_response.find('{')
        end_idx = raw_response.rfind('}')
        
        if start_idx != -1 and end_idx != -1:
            clean_json = raw_response[start_idx:end_idx+1]
        else:
            clean_json = raw_response
        
        # Replace Arabic commas with English commas
        clean_json = clean_json.replace('،', ',')
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(clean_json)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            print(f"Problematic JSON: {clean_json[:500]}...")
            # Return raw response for debugging
            return clean_json

    def validate_conversation_structure(self, structure_json):
        """
        Validate the conversation structure
        """
        required_keys = ["episode_topic", "personas", "conversation_flow", "cultural_context", "language_style", "technical_notes"]
        
        conversation_flow_required = ["intro1", "intro2", "main_discussion", "closing"]
        intro1_required = ["opening_line", "podcast_introduction", "episode_hook"]
        intro2_required = ["topic_introduction", "guest_welcome", "guest_bio_highlight"]
        
        try:
            import json
            structure = json.loads(structure_json)
            
            missing_keys = []
            
            # Check main structure
            for key in required_keys:
                if key not in structure:
                    missing_keys.append(key)
            
            # Check conversation flow
            if "conversation_flow" in structure:
                conv_flow = structure["conversation_flow"]
                for key in conversation_flow_required:
                    if key not in conv_flow:
                        missing_keys.append(f"conversation_flow.{key}")
                
                # Check intro1
                if "intro1" in conv_flow:
                    intro1 = conv_flow["intro1"]
                    for key in intro1_required:
                        if key not in intro1:
                            missing_keys.append(f"intro1.{key}")
                
                # Check intro2
                if "intro2" in conv_flow:
                    intro2 = conv_flow["intro2"]
                    for key in intro2_required:
                        if key not in intro2:
                            missing_keys.append(f"intro2.{key}")
                
                # Check main discussion
                if "main_discussion" in conv_flow:
                    main_disc = conv_flow["main_discussion"]
                    if not isinstance(main_disc, list) or len(main_disc) < 3:
                        missing_keys.append("main_discussion (need at least 3 points)")
                    else:
                        for i, point in enumerate(main_disc):
                            if "point_title" not in point:
                                missing_keys.append(f"main_discussion[{i}].point_title")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Conversation structure validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_structure_quality(self, structure_json):
        """
        Analyze the quality of the conversation structure
        """
        try:
            import json
            structure = json.loads(structure_json)
            
            analysis = {}
            
            # Check completeness
            conv_flow = structure.get("conversation_flow", {})
            analysis["has_intro1"] = bool(conv_flow.get("intro1"))
            analysis["has_intro2"] = bool(conv_flow.get("intro2"))
            analysis["has_main_discussion"] = bool(conv_flow.get("main_discussion"))
            analysis["has_closing"] = bool(conv_flow.get("closing"))
            
            # Check main discussion depth
            main_disc = conv_flow.get("main_discussion", [])
            analysis["discussion_points"] = len(main_disc)
            analysis["adequate_points"] = len(main_disc) >= 3
            
            # Check cultural context
            cultural = structure.get("cultural_context", {})
            analysis["has_proverbs"] = len(cultural.get("proverbs_sayings", [])) >= 1
            analysis["has_regional_refs"] = len(cultural.get("regional_references", [])) >= 1
            
            # Overall readiness
            readiness_indicators = [
                analysis["has_intro1"],
                analysis["has_intro2"],
                analysis["has_main_discussion"],
                analysis["has_closing"],
                analysis["adequate_points"],
                analysis["has_proverbs"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_next_step"] = analysis["readiness_score"] >= 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze structure quality"}

# Usage:
# generator = ConversationStructureGenerator(deployment, "Fanar-C-1-8.7B") 
# structure_result = generator.generate_conversation_structure(topic, information, classification_result, personas_result)

In [13]:
generator = ConversationStructureGenerator(deployment, "Fanar-C-1-8.7B")
outline_result = generator.generate_conversation_structure(topic, information, classification_result, persona_result)

print("Conversation structure Result:")
print(outline_result)

try:
    parsed_result = json.loads(outline_result)
    print("✅ JSON parsed successfully")
    print(f"Episode topic: {parsed_result.get('episode_topic', 'Not found')}")
except json.JSONDecodeError as e:
    print(f"❌ Error parsing JSON: {e}")
    print("First 200 chars of response:")
    print(outline_result[:200])

Conversation structure Result:
{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "لمى عبد الله",
      "background": "صحفية متخصصة في الشأن التكنولوجي في إحدى الجرائد المحلية",
      "speaking_style": "تطرح أسئلة مفتوحة لتعميق النقاش وتعزز الحديث بموضوعية واضحة"
    },
    "guest": {
      "name": "علي محسن",
      "background": "باحث في علوم الكمبيوتر وأستاذ مساعد في جامعة حكومية",
      "speaking_style": "يعرض أفكارًا فنية بأسلوب سلس ويتفاعل مع المواضيع بتحليل عميق"
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحباً جميعاً! اليوم نناقش دور الذكاء الاصطناعي في صون هويتنا العربية.",
      "podcast_introduction": "هذا برنامج 'العالم بين يديك' حيث نبحث في تحديات المجتمع المعاصر.",
      "episode_hook": "هل يمكن للآلات الفكرية فهم روح مجتمعاتنا؟ دعونا نعرف!",
      "tone_guidance": "تحليلي ومتفتح للأفكار"
    },
    "intro2": {
      "topic_introduction": "نتناول هنا مح

In [25]:
import json
import time

class SectionalDialogueContentEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_intro_sections(self, topic, classification_result, personas_result, intro1, intro2):
        """
        Chunk 1: Enhance intro1 and intro2 sections only
        
        Args:
            topic: Main topic
            classification_result: JSON string from classification
            personas_result: JSON string from personas
            intro1: intro1 section from conversation_flow
            intro2: intro2 section from conversation_flow
            
        Returns:
            Enhanced intro1 and intro2 sections
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        primary_category = classification.get("primary_category", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
You are an expert in enhancing Arabic podcast introductions.

Task: Enhance ONLY the intro sections with natural dialogue elements.

Topic: {topic}
Category: {primary_category}
Style: {optimal_style}

Host: {host_name} - {host.get('background', '')}
Guest: {guest_name} - {guest.get('background', '')}

Current intro1: {json.dumps(intro1, ensure_ascii=False)}
Current intro2: {json.dumps(intro2, ensure_ascii=False)}

ENHANCEMENT REQUIREMENTS:

For intro1, ADD these fields:
- "spontaneity_elements": [3-4 natural spontaneous phrases that the host might use when opening, in MSA]

For intro2, ADD these fields:  
- "cultural_connections": [2-3 ways to connect this topic to Arab culture/values, in MSA]

Return the enhanced sections in this exact format:
{{
    "intro1": {{
        [keep all existing intro1 fields],
        "spontaneity_elements": [new content]
    }},
    "intro2": {{
        [keep all existing intro2 fields],
        "cultural_connections": [new content]
    }}
}}

CRITICAL REQUIREMENTS:
- Keep ALL existing content unchanged
- Add only the specified new fields
- All new values in Modern Standard Arabic (MSA)
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
- Make content specific to topic: {topic}
- Match the {optimal_style} style
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You enhance Arabic podcast intros. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )

        return self._clean_json_response(response.choices[0].message.content)

    def enhance_main_discussion_point(self, topic, classification_result, personas_result, discussion_point, point_index):
        """
        Chunk 2: Enhance individual main discussion points
        
        Args:
            topic: Main topic
            classification_result: JSON string from classification  
            personas_result: JSON string from personas
            discussion_point: Single discussion point to enhance
            point_index: Index of this point (for logging)
            
        Returns:
            Enhanced discussion point
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in enhancing Arabic podcast discussion points.

Task: Enhance ONE discussion point with rich dialogue elements.

Topic: {topic}
Style: {optimal_style}
Point #{point_index + 1}

Current discussion point: {json.dumps(discussion_point, ensure_ascii=False)}

Add EXACTLY these 5 fields. Keep all existing fields unchanged:

{{
    [all existing fields from discussion_point],
    "spontaneous_triggers": ["trigger 1 in MSA", "trigger 2 in MSA"],
    "disagreement_points": "disagreement description in MSA",
    "cultural_references": ["reference 1 in MSA", "reference 2 in MSA"],
    "natural_transitions": "transition phrase in MSA",
    "emotional_triggers": "emotional description in MSA"
}}

CRITICAL REQUIREMENTS:
- Keep ALL existing fields exactly as they are
- Add only the 5 new fields shown above
- All new content in Modern Standard Arabic (MSA)
- Use English punctuation ONLY (no ،)
- Return only valid JSON, no extra text
- Make content relevant to topic: {topic}
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You enhance Arabic podcast discussion points. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def enhance_closing_sections(self, topic, classification_result, personas_result, closing_section):
        """
        Chunk 3: Enhance closing (conclusion + outro) sections
        
        Args:
            topic: Main topic
            classification_result: JSON string from classification
            personas_result: JSON string from personas  
            closing_section: closing section from conversation_flow
            
        Returns:
            Enhanced closing section
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in enhancing Arabic podcast closings.

Task: Enhance the closing section with natural wrap-up elements.

Topic: {topic}
Style: {optimal_style}

Host: {host_name} - {host.get('background', '')}
Guest: {guest_name} - {guest.get('background', '')}

Current closing: {json.dumps(closing_section, ensure_ascii=False)}

ENHANCEMENT REQUIREMENTS:

For conclusion subsection, ADD:
- "emotional_closure": "how to create emotional satisfaction for listeners, in MSA"
- "key_insights": [2-3 key insights that should be highlighted in the wrap-up, in MSA]

For outro subsection, ADD:
- "memorable_ending": "a memorable way to end that listeners will remember, in MSA"
- "connection_building": "ways to build ongoing connection with the audience, in MSA"

Return enhanced closing in this exact format:
{{
    "conclusion": {{
        [keep all existing conclusion fields],
        "emotional_closure": "new content",
        "key_insights": [new content]
    }},
    "outro": {{
        [keep all existing outro fields],
        "memorable_ending": "new content",
        "connection_building": "new content"
    }}
}}

CRITICAL REQUIREMENTS:
- Keep ALL existing content unchanged
- Add only the specified new fields
- All new values in Modern Standard Arabic (MSA)
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
- Make content feel conclusive and satisfying
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You enhance Arabic podcast closings. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def create_global_elements(self, topic, classification_result, personas_result):
        """
        Chunk 4: Create global elements (spontaneous_moments, personality_interactions, dialogue_techniques)
        
        Args:
            topic: Main topic
            classification_result: JSON string from classification
            personas_result: JSON string from personas
            
        Returns:
            Global elements as JSON string
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        primary_category = classification.get("primary_category", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in creating global dialogue elements for Arabic podcasts.

Task: Create three global sections that enhance the overall conversation flow.

Topic: {topic}
Category: {primary_category}
Style: {optimal_style}

Host: {host_name} - {host.get('background', '')}
Guest: {guest_name} - {guest.get('background', '')}

Create EXACTLY this JSON structure with proper English punctuation:

{{
    "spontaneous_moments": {{
        "natural_interruptions": [
            "first natural interruption in MSA",
            "second natural interruption in MSA",
            "third natural interruption in MSA"
        ],
        "emotional_reactions": [
            "first emotional reaction in MSA",
            "second emotional reaction in MSA", 
            "third emotional reaction in MSA"
        ],
        "personal_stories": [
            "first personal story in MSA",
            "second personal story in MSA"
        ],
        "humorous_moments": [
            "first humorous moment in MSA",
            "second humorous moment in MSA"
        ]
    }},
    "personality_interactions": {{
        "host_strengths": "host strengths description in MSA",
        "guest_expertise": "guest expertise description in MSA",
        "natural_chemistry": "chemistry description in MSA",
        "tension_points": "tension points description in MSA",
        "collaboration_moments": "collaboration description in MSA"
    }},
    "dialogue_techniques": {{
        "questioning_styles": [
            "first questioning style in MSA",
            "second questioning style in MSA",
            "third questioning style in MSA"
        ],
        "storytelling_moments": [
            "first storytelling moment in MSA",
            "second storytelling moment in MSA"
        ],
        "audience_engagement": [
            "first engagement technique in MSA",
            "second engagement technique in MSA",
            "third engagement technique in MSA"
        ],
        "emotional_peaks": [
            "first emotional peak in MSA",
            "second emotional peak in MSA"
        ]
    }}
}}

CRITICAL REQUIREMENTS:
- Replace placeholder text with actual content in Modern Standard Arabic (MSA)
- Use ONLY English commas (,) and standard quotes (")
- NO Arabic commas (،) or special punctuation
- NO extra text before or after JSON
- NO explanatory text
- Make content specific to {host_name}, {guest_name}, and topic: {topic}
- Follow the EXACT structure shown above
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You create global dialogue elements. Return ONLY valid JSON with English punctuation. No Arabic commas. No extra text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7  # Reduced temperature for better JSON compliance
        )

        return self._clean_json_response(response.choices[0].message.content)

    def enhance_dialogue_content(self, topic, information, classification_result, personas_result, structure_result):
        """
        Main orchestration method: Coordinates all chunks
        """
        print("🔧 Starting sectional dialogue enhancement...")
        print("=" * 50)
        
        try:
            structure = json.loads(structure_result)
        except:
            raise ValueError("Invalid structure JSON provided")
        
        # Extract sections
        conv_flow = structure.get("conversation_flow", {})
        intro1 = conv_flow.get("intro1", {})
        intro2 = conv_flow.get("intro2", {})
        main_discussion = conv_flow.get("main_discussion", [])
        closing = conv_flow.get("closing", {})
        
        # Chunk 1: Enhance intro sections
        print("📝 Chunk 1: Enhancing intro sections...")
        try:
            enhanced_intros_json = self.enhance_intro_sections(
                topic, classification_result, personas_result, intro1, intro2
            )
            enhanced_intros = json.loads(enhanced_intros_json)
            
            # Update structure
            structure["conversation_flow"]["intro1"].update(enhanced_intros.get("intro1", {}))
            structure["conversation_flow"]["intro2"].update(enhanced_intros.get("intro2", {}))
            print("✅ Intro sections enhanced successfully")
            
        except Exception as e:
            print(f"⚠️ Error enhancing intros: {e}")
        
        # Small delay between chunks
        time.sleep(1)
        
        # Chunk 2: Enhance main discussion points (one by one)
        print("📝 Chunk 2: Enhancing main discussion points...")
        enhanced_discussion_points = []
        
        for i, point in enumerate(main_discussion):
            print(f"  Enhancing discussion point {i+1}/{len(main_discussion)}...")
            try:
                enhanced_point_json = self.enhance_main_discussion_point(
                    topic, classification_result, personas_result, point, i
                )
                enhanced_point = json.loads(enhanced_point_json)
                enhanced_discussion_points.append(enhanced_point)
                print(f"  ✅ Point {i+1} enhanced successfully")
                
                # Small delay between points
                time.sleep(0.5)
                
            except Exception as e:
                print(f"  ⚠️ Error enhancing point {i+1}: {e}")
                print(f"  🔄 Attempting fallback enhancement for point {i+1}...")
                
                # Try fallback enhancement for this point
                try:
                    fallback_point = self._create_fallback_discussion_point(
                        topic, classification_result, personas_result, point, i
                    )
                    enhanced_discussion_points.append(fallback_point)
                    print(f"  ✅ Point {i+1} enhanced with fallback method")
                except Exception as fallback_error:
                    print(f"  ⚠️ Fallback failed for point {i+1}: {fallback_error}")
                    # Add minimal enhancements to original point
                    enhanced_point = point.copy()
                    enhanced_point.update(self._get_minimal_discussion_enhancements())
                    enhanced_discussion_points.append(enhanced_point)
                    print(f"  📝 Point {i+1} enhanced with minimal defaults")
        
        # Update structure with enhanced discussion points
        structure["conversation_flow"]["main_discussion"] = enhanced_discussion_points
        print("✅ All main discussion points processed")
        
        # Small delay between chunks
        time.sleep(1)
        
        # Chunk 3: Enhance closing sections
        print("📝 Chunk 3: Enhancing closing sections...")
        try:
            enhanced_closing_json = self.enhance_closing_sections(
                topic, classification_result, personas_result, closing
            )
            enhanced_closing = json.loads(enhanced_closing_json)
            
            # Update structure
            structure["conversation_flow"]["closing"].update(enhanced_closing)
            print("✅ Closing sections enhanced successfully")
            
        except Exception as e:
            print(f"⚠️ Error enhancing closing: {e}")
        
        # Small delay between chunks  
        time.sleep(1)
        
        # Chunk 4: Create global elements
        print("📝 Chunk 4: Creating global elements...")
        try:
            global_elements_json = self.create_global_elements(
                topic, classification_result, personas_result
            )
            global_elements = json.loads(global_elements_json)
            
            # Add global elements to structure
            structure.update(global_elements)
            print("✅ Global elements created successfully")
            
        except Exception as e:
            print(f"⚠️ Error creating global elements: {e}")
            print("🔄 Attempting to create fallback global elements...")
            
            # Create fallback global elements
            try:
                fallback_elements = self._create_fallback_global_elements(
                    topic, classification_result, personas_result
                )
                structure.update(fallback_elements)
                print("✅ Fallback global elements created successfully")
            except Exception as fallback_error:
                print(f"⚠️ Fallback also failed: {fallback_error}")
                print("📝 Using minimal default global elements...")
                # Add minimal default elements so validation doesn't fail
                structure.update(self._get_minimal_global_elements())
        
        print("=" * 50)
        print("🎉 Sectional dialogue enhancement completed!")
        
        return json.dumps(structure, ensure_ascii=False, indent=2)

    def _clean_json_response(self, response):
        """Helper method to clean JSON response - Enhanced version"""
        response = response.strip()
        
        # Remove any text before first { and after last }
        start_idx = response.find('{')
        end_idx = response.rfind('}')
        
        if start_idx != -1 and end_idx != -1:
            clean_json = response[start_idx:end_idx+1]
        else:
            clean_json = response
        
        # Replace Arabic commas and punctuation with English equivalents
        clean_json = clean_json.replace('،', ',')
        clean_json = clean_json.replace('"', '"')
        clean_json = clean_json.replace('"', '"')
        clean_json = clean_json.replace(''', "'")
        clean_json = clean_json.replace(''', "'")
        
        # Fix common JSON issues
        # Remove trailing commas before closing braces/brackets
        import re
        clean_json = re.sub(r',(\s*[}\]])', r'\1', clean_json)
        
        # Ensure proper quote escaping
        clean_json = clean_json.replace('\\"', '"')
        
        return clean_json

    def validate_enhanced_content(self, enhanced_json):
        """Validate the enhanced dialogue content"""
        try:
            enhanced = json.loads(enhanced_json)
            
            missing_elements = []
            
            # Check global sections
            required_global = ["spontaneous_moments", "personality_interactions", "dialogue_techniques"]
            for element in required_global:
                if element not in enhanced:
                    missing_elements.append(element)
            
            # Check enhanced conversation flow
            conv_flow = enhanced.get("conversation_flow", {})
            
            # Check intro1 enhancements
            intro1 = conv_flow.get("intro1", {})
            if "spontaneity_elements" not in intro1:
                missing_elements.append("intro1.spontaneity_elements")
            
            # Check intro2 enhancements  
            intro2 = conv_flow.get("intro2", {})
            if "cultural_connections" not in intro2:
                missing_elements.append("intro2.cultural_connections")
            
            # Check main discussion enhancements
            main_disc = conv_flow.get("main_discussion", [])
            required_point_fields = ["spontaneous_triggers", "disagreement_points", "cultural_references", "natural_transitions", "emotional_triggers"]
            
            for i, point in enumerate(main_disc):
                for field in required_point_fields:
                    if field not in point:
                        missing_elements.append(f"main_discussion[{i}].{field}")
            
            # Check closing enhancements
            closing = conv_flow.get("closing", {})
            conclusion = closing.get("conclusion", {})
            outro = closing.get("outro", {})
            
            if "emotional_closure" not in conclusion:
                missing_elements.append("closing.conclusion.emotional_closure")
            if "key_insights" not in conclusion:
                missing_elements.append("closing.conclusion.key_insights")
            if "memorable_ending" not in outro:
                missing_elements.append("closing.outro.memorable_ending")
            if "connection_building" not in outro:
                missing_elements.append("closing.outro.connection_building")
            
            if missing_elements:
                return False, f"Missing enhanced elements: {missing_elements}"
            
            return True, "Sectional dialogue content enhancement validation successful"
            
        except json.JSONDecodeError:
            def _create_fallback_global_elements(self, topic, classification_result, personas_result):
                    """Create fallback global elements with simpler prompts"""
                    try:
                        classification = json.loads(classification_result)
                        personas = json.loads(personas_result)
                    except:
                        raise ValueError("Invalid JSON provided")
                    
                    host = personas.get("host", {})
                    guest = personas.get("guest", {})
                    host_name = host.get('name', 'المقدم')
                    guest_name = guest.get('name', 'الضيف')
                    
                    # Create each section separately with very simple prompts
                    fallback_elements = {}
                    
                    # Spontaneous moments
                    try:
                        spont_prompt = f'Create spontaneous_moments for topic "{topic}" with host {host_name} and guest {guest_name}. Return only JSON with natural_interruptions, emotional_reactions, personal_stories, humorous_moments arrays. All values in MSA.'
                        
                        spont_response = self.deployment.chat.completions.create(
                            model=self.model,
                            messages=[{"role": "user", "content": spont_prompt}],
                            temperature=0.6
                        )
                        spont_json = self._clean_json_response(spont_response.choices[0].message.content)
                        fallback_elements["spontaneous_moments"] = json.loads(spont_json)
                    except:
                        fallback_elements["spontaneous_moments"] = self._get_default_spontaneous_moments()
                    
                    # Personality interactions  
                    try:
                        personality_prompt = f'Create personality_interactions for {host_name} and {guest_name} discussing "{topic}". Return only JSON with host_strengths, guest_expertise, natural_chemistry, tension_points, collaboration_moments. All values in MSA.'
                        
                        personality_response = self.deployment.chat.completions.create(
                            model=self.model,
                            messages=[{"role": "user", "content": personality_prompt}],
                            temperature=0.6
                        )
                        personality_json = self._clean_json_response(personality_response.choices[0].message.content)
                        fallback_elements["personality_interactions"] = json.loads(personality_json)
                    except:
                        fallback_elements["personality_interactions"] = self._get_default_personality_interactions(host_name, guest_name)
                    
                    # Dialogue techniques
                    try:
                        dialogue_prompt = f'Create dialogue_techniques for Arabic podcast about "{topic}". Return only JSON with questioning_styles, storytelling_moments, audience_engagement, emotional_peaks arrays. All values in MSA.'
                        
                        dialogue_response = self.deployment.chat.completions.create(
                            model=self.model,
                            messages=[{"role": "user", "content": dialogue_prompt}],
                            temperature=0.6
                        )
                        dialogue_json = self._clean_json_response(dialogue_response.choices[0].message.content)
                        fallback_elements["dialogue_techniques"] = json.loads(dialogue_json)
                    except:
                        fallback_elements["dialogue_techniques"] = self._get_default_dialogue_techniques()
                    
                    return fallback_elements

    def _get_minimal_global_elements(self):
        """Return minimal default global elements"""
        return {
            "spontaneous_moments": self._get_default_spontaneous_moments(),
            "personality_interactions": self._get_default_personality_interactions("المقدم", "الضيف"),
            "dialogue_techniques": self._get_default_dialogue_techniques()
        }

    def _get_default_spontaneous_moments(self):
        """Default spontaneous moments"""
        return {
            "natural_interruptions": [
                "اسمحي لي أن أضيف نقطة هنا",
                "هذا يذكرني بموقف مشابه",
                "انتظر، هذا مهم جداً"
            ],
            "emotional_reactions": [
                "هذا مؤثر فعلاً",
                "لم أفكر في الأمر من هذه الزاوية",
                "أتفق معك تماماً"
            ],
            "personal_stories": [
                "أتذكر موقفاً مشابهاً حدث معي",
                "في تجربتي الشخصية وجدت أن"
            ],
            "humorous_moments": [
                "هذا يذكرني بنكتة لطيفة",
                "الموقف له جانب طريف"
            ]
        }

    def _create_fallback_discussion_point(self, topic, classification_result, personas_result, discussion_point, point_index):
        """Create fallback enhancement for a single discussion point"""
        try:
            classification = json.loads(classification_result)
        except:
            classification = {}
        
        # Create a simple enhancement with minimal API call
        enhanced_point = discussion_point.copy()
        
        # Add minimal enhancements
        enhanced_point.update({
            "spontaneous_triggers": [
                "هذا يثير تساؤلاً مهماً",
                "دعني أشارككم تجربة في هذا المجال"
            ],
            "disagreement_points": "قد تختلف وجهات النظر حول أفضل طريقة للتعامل مع هذه القضية",
            "cultural_references": [
                "كما يقول المثل: العلم نور",
                "تراثنا يعلمنا أهمية التوازن في كل شيء"
            ],
            "natural_transitions": "هذا يقودنا إلى نقطة مهمة أخرى",
            "emotional_triggers": "هذا الموضوع يلامس قلوب كل من يهتم بمستقبل ثقافتنا"
        })
        
        return enhanced_point

    def _get_minimal_discussion_enhancements(self):
        """Get minimal enhancements for discussion points"""
        return {
            "spontaneous_triggers": ["هذا مهم فعلاً", "دعني أضيف شيئاً"],
            "disagreement_points": "قد نختلف في وجهات النظر",
            "cultural_references": ["العلم نور", "الحكمة ضالة المؤمن"],
            "natural_transitions": "لننتقل للنقطة التالية",
            "emotional_triggers": "هذا يثير مشاعر قوية"
        }

    def _create_fallback_global_elements(self, topic, classification_result, personas_result):
        """Create fallback global elements with simpler prompts"""
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            return self._get_minimal_global_elements()
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        # Create each section separately with very simple prompts
        fallback_elements = {}
        
        # Spontaneous moments
        try:
            spont_prompt = f'Create JSON for spontaneous_moments about "{topic}". Include natural_interruptions, emotional_reactions, personal_stories, humorous_moments arrays. All in MSA. Use English punctuation only.'
            
            spont_response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only valid JSON with English punctuation."},
                    {"role": "user", "content": spont_prompt}
                ],
                temperature=0.5
            )
            spont_json = self._clean_json_response(spont_response.choices[0].message.content)
            fallback_elements["spontaneous_moments"] = json.loads(spont_json)
        except Exception as e:
            print(f"    Spontaneous moments fallback failed: {e}")
            fallback_elements["spontaneous_moments"] = self._get_default_spontaneous_moments()
        
        # Personality interactions  
        try:
            personality_prompt = f'Create JSON for personality_interactions between host {host_name} and guest {guest_name}. Include host_strengths, guest_expertise, natural_chemistry, tension_points, collaboration_moments. All in MSA. Use English punctuation only.'
            
            personality_response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only valid JSON with English punctuation."},
                    {"role": "user", "content": personality_prompt}
                ],
                temperature=0.5
            )
            personality_json = self._clean_json_response(personality_response.choices[0].message.content)
            fallback_elements["personality_interactions"] = json.loads(personality_json)
        except Exception as e:
            print(f"    Personality interactions fallback failed: {e}")
            fallback_elements["personality_interactions"] = self._get_default_personality_interactions(host_name, guest_name)
        
        # Dialogue techniques
        try:
            dialogue_prompt = f'Create JSON for dialogue_techniques for podcast about "{topic}". Include questioning_styles, storytelling_moments, audience_engagement, emotional_peaks arrays. All in MSA. Use English punctuation only.'
            
            dialogue_response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only valid JSON with English punctuation."},
                    {"role": "user", "content": dialogue_prompt}
                ],
                temperature=0.5
            )
            dialogue_json = self._clean_json_response(dialogue_response.choices[0].message.content)
            fallback_elements["dialogue_techniques"] = json.loads(dialogue_json)
        except Exception as e:
            print(f"    Dialogue techniques fallback failed: {e}")
            fallback_elements["dialogue_techniques"] = self._get_default_dialogue_techniques()
        
        return fallback_elements

# Usage:
# enhancer = SectionalDialogueContentEnhancer(deployment, "Fanar-C-1-8.7B")
# enhanced_result = enhancer.enhance_dialogue_content(topic, information, classification_result, persona_result, outline_result)

    def _get_default_personality_interactions(self, host_name, guest_name):
        """Default personality interactions"""
        return {
            "host_strengths": f"{host_name} ماهر في طرح الأسئلة المناسبة وتوجيه الحوار",
            "guest_expertise": f"{guest_name} يقدم معرفة عميقة في مجال تخصصه",
            "natural_chemistry": "يتفاعل المقدم والضيف بطريقة طبيعية ومريحة",
            "tension_points": "قد يختلفان في بعض وجهات النظر مما يثري النقاش",
            "collaboration_moments": "يبنيان على أفكار بعضهما البعض لإثراء المحتوى"
        }

    def _get_default_dialogue_techniques(self):
        """Default dialogue techniques"""
        return {
            "questioning_styles": [
                "أسئلة مفتوحة لتعميق النقاش",
                "أسئلة تحليلية للوصول للجذور",
                "أسئلة شخصية لإضافة البعد الإنساني"
            ],
            "storytelling_moments": [
                "سرد تجارب شخصية ذات صلة",
                "قصص نجاح ملهمة"
            ],
            "audience_engagement": [
                "طرح أسئلة يفكر فيها المستمع",
                "استخدام أمثلة من الواقع",
                "دعوة المستمعين للتفاعل"
            ],
            "emotional_peaks": [
                "لحظات تأملية عميقة",
                "قصص مؤثرة تلامس القلب"
            ]
        }

In [26]:
# This should now work without the global elements error
enhancer = SectionalDialogueContentEnhancer(deployment, "Fanar-C-1-8.7B")
enhanced_result = enhancer.enhance_dialogue_content(
    topic, 
    information, 
    classification_result, 
    persona_result, 
    outline_result
)

print("Enhanced dialogue content:")
print(enhanced_result)

# Validation should now pass
is_valid, message = enhancer.validate_enhanced_content(enhanced_result)
print(f"Validation: {message}")

🔧 Starting sectional dialogue enhancement...
📝 Chunk 1: Enhancing intro sections...
✅ Intro sections enhanced successfully
📝 Chunk 2: Enhancing main discussion points...
  Enhancing discussion point 1/3...
  ⚠️ Error enhancing point 1: Expecting ',' delimiter: line 4 column 77 (char 228)
  🔄 Attempting fallback enhancement for point 1...
  ✅ Point 1 enhanced with fallback method
  Enhancing discussion point 2/3...
  ✅ Point 2 enhanced successfully
  Enhancing discussion point 3/3...
  ✅ Point 3 enhanced successfully
✅ All main discussion points processed
📝 Chunk 3: Enhancing closing sections...
✅ Closing sections enhanced successfully
📝 Chunk 4: Creating global elements...
⚠️ Error creating global elements: Expecting ',' delimiter: line 6 column 87 (char 366)
🔄 Attempting to create fallback global elements...
✅ Fallback global elements created successfully
🎉 Sectional dialogue enhancement completed!
Enhanced dialogue content:
{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف 

In [27]:
class EnhancedFinalPolishEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def polish_spontaneous_moments(self, topic, classification_result, personas_result, current_spontaneous_moments):
        """
        Chunk 1: Polish and expand spontaneous moments
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in refining spontaneous conversation elements for Arabic podcasts.

Task: Enhance and expand the existing spontaneous_moments section.

Topic: {topic}
Style: {optimal_style}
Host: {host_name} - {host.get('background', '')}
Guest: {guest_name} - {guest.get('background', '')}

Current spontaneous moments: {json.dumps(current_spontaneous_moments, ensure_ascii=False)}

Expand this section with more detailed, character-specific content:

{{
    "natural_interruptions": [5-6 specific interruptions that {host_name} and {guest_name} might naturally make, in MSA],
    "emotional_reactions": [5-6 authentic emotional reactions specific to this topic and these personalities, in MSA],
    "personal_stories": [3-4 personal stories based on their actual backgrounds and the topic, in MSA],
    "humorous_moments": [3-4 light humor moments that fit the style and topic perfectly, in MSA],
    "spontaneous_questions": [3-4 unexpected questions that might arise naturally during discussion, in MSA],
    "connecting_phrases": [4-5 natural connecting phrases for smooth conversation flow, in MSA]
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make content specific to {host_name}, {guest_name}, and topic: {topic}
- Match the {optimal_style} conversation style
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You refine spontaneous elements for Arabic podcasts. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )

        return self._clean_json_response(response.choices[0].message.content)

    def polish_personality_interactions(self, topic, classification_result, personas_result, current_personality_interactions):
        """
        Chunk 2: Polish and deepen personality interactions
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in refining personality dynamics for Arabic podcasts.

Task: Enhance and deepen the personality_interactions section.

Topic: {topic}
Style: {optimal_style}
Host: {host_name} - {host.get('background', '')} - {host.get('speaking_style', '')}
Guest: {guest_name} - {guest.get('background', '')} - {guest.get('speaking_style', '')}

Current personality interactions: {json.dumps(current_personality_interactions, ensure_ascii=False)}

Create enhanced personality interactions:

{{
    "host_strengths": "detailed description of {host_name}'s specific conversational strengths for this topic, in MSA",
    "guest_expertise": "detailed description of {guest_name}'s expertise and how it shines in this discussion, in MSA",
    "natural_chemistry": "specific ways {host_name} and {guest_name} complement each other naturally, in MSA",
    "tension_points": "specific healthy disagreement areas that create engaging dialogue, in MSA",
    "collaboration_moments": "specific moments where they build beautifully on each other's ideas, in MSA",
    "communication_styles": "how their different communication styles create dynamic dialogue, in MSA",
    "mutual_respect": "how they show respect for each other's viewpoints during discussion, in MSA",
    "energy_dynamics": "how their energy levels and enthusiasm interact throughout the conversation, in MSA"
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make descriptions specific to these exact personalities and topic
- Focus on realistic, believable interactions
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You refine personality dynamics. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def create_shared_experiences(self, topic, classification_result, personas_result):
        """
        Chunk 3: Create shared experiences section
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        primary_category = classification.get("primary_category", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in identifying shared experiences for Arabic podcast conversations.

Task: Create a shared_experiences section for meaningful connections.

Topic: {topic}
Category: {primary_category}
Style: {optimal_style}

Host: {host_name} - {host.get('background', '')}
Guest: {guest_name} - {guest.get('background', '')}

Create this exact structure:

{{
    "common_ground": [
        "first specific similarity between {host_name} and {guest_name} related to topic, in MSA",
        "second specific similarity that creates connection, in MSA", 
        "third common experience they can relate to, in MSA"
    ],
    "generational_perspectives": [
        "first generational difference that enriches discussion, in MSA",
        "second age-related viewpoint difference, in MSA",
        "third perspective difference based on experience, in MSA"
    ],
    "professional_overlaps": [
        "first area where their work/interests intersect, in MSA",
        "second professional connection point, in MSA"
    ],
    "cultural_touchstones": [
        "first shared cultural reference they both understand, in MSA",
        "second cultural element that connects them, in MSA",
        "third cultural experience they can both relate to, in MSA"
    ]
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make connections specific to these personalities and topic
- Create believable, realistic shared experiences
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You create shared experiences. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def create_contemporary_relevance(self, topic, classification_result, personas_result):
        """
        Chunk 4: Create contemporary relevance section
        """
        try:
            classification = json.loads(classification_result)
        except:
            classification = {}
        
        optimal_style = classification.get("optimal_style", "")
        primary_category = classification.get("primary_category", "")

        prompt = f"""
You are an expert in connecting topics to contemporary relevance for Arabic podcasts.

Task: Create contemporary_relevance section connecting the topic to current times.

Topic: {topic}
Category: {primary_category}
Style: {optimal_style}

Create this exact structure:

{{
    "current_events": [
        "first current event related to {topic} in the Arab world, in MSA",
        "second recent development relevant to the topic, in MSA",
        "third contemporary happening that connects to the discussion, in MSA"
    ],
    "future_implications": [
        "first future impact of {topic} on Arab society, in MSA", 
        "second long-term implication to consider, in MSA",
        "third future trend or development to watch, in MSA"
    ],
    "regional_perspectives": [
        "how {topic} affects different Arab regions differently, in MSA",
        "regional approaches or responses to this topic, in MSA"
    ],
    "global_connections": [
        "how {topic} connects Arab world to global trends, in MSA",
        "international aspects that affect Arab perspectives, in MSA"
    ]
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make content specific to topic: {topic}
- Focus on realistic, current relevance
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You create contemporary relevance. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def enhance_cultural_context(self, topic, classification_result, current_cultural_context):
        """
        Chunk 5: Enhance cultural context with more depth
        """
        try:
            classification = json.loads(classification_result)
        except:
            classification = {}
        
        optimal_style = classification.get("optimal_style", "")
        primary_category = classification.get("primary_category", "")

        prompt = f"""
You are an expert in enhancing cultural context for Arabic podcasts.

Task: Enhance and expand the cultural_context section.

Topic: {topic}
Category: {primary_category}
Current cultural context: {json.dumps(current_cultural_context, ensure_ascii=False)}

Create enhanced cultural context:

{{
    "proverbs_sayings": [
        "first relevant Arabic proverb for {topic}, in MSA",
        "second wise saying that applies to the discussion, in MSA",
        "third cultural wisdom relevant to the topic, in MSA",
        "fourth proverb that adds depth to the conversation, in MSA"
    ],
    "regional_references": [
        "first regional reference relevant to {topic}, in MSA",
        "second local example or story, in MSA", 
        "third cultural reference from Arab history, in MSA",
        "fourth contemporary Arab example, in MSA"
    ],
    "historical_connections": [
        "first historical parallel or lesson relevant to {topic}, in MSA",
        "second way this topic connects to Arab heritage, in MSA"
    ],
    "literary_references": [
        "first relevant quote from Arabic literature, in MSA",
        "second literary reference that enriches the discussion, in MSA"
    ]
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make all references authentic and relevant to topic
- Include accurate cultural knowledge
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You enhance cultural context. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )

        return self._clean_json_response(response.choices[0].message.content)

    def create_advanced_dialogue_flow(self, topic, classification_result, personas_result):
        """
        Chunk 6: Create advanced dialogue flow section for smoother conversations
        """
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        optimal_style = classification.get("optimal_style", "")
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')

        prompt = f"""
You are an expert in creating advanced dialogue flow for Arabic podcasts.

Task: Create an advanced_dialogue_flow section for natural conversation dynamics.

Topic: {topic}
Style: {optimal_style}
Host: {host_name}
Guest: {guest_name}

Create this structure:

{{
    "conversation_rhythms": [
        "first natural rhythm pattern for this type of discussion, in MSA",
        "second pacing technique that works well, in MSA",
        "third rhythm variation to maintain interest, in MSA"
    ],
    "bridge_phrases": [
        "first smooth transition phrase {host_name} might use, in MSA",
        "second connecting phrase for topic shifts, in MSA", 
        "third natural bridge between ideas, in MSA"
    ],
    "emphasis_techniques": [
        "first way to emphasize important points naturally, in MSA",
        "second technique for highlighting key insights, in MSA",
        "third method for creating memorable moments, in MSA"
    ],
    "recovery_strategies": [
        "first way to gracefully recover from awkward moments, in MSA",
        "second technique for handling disagreements smoothly, in MSA"
    ],
    "engagement_boosters": [
        "first technique to re-engage if energy drops, in MSA",
        "second way to maintain listener interest, in MSA",
        "third method for creating connection with audience, in MSA"
    ]
}}

CRITICAL REQUIREMENTS:
- All content in Modern Standard Arabic (MSA)
- Make techniques specific to these personalities and topic
- Focus on practical, usable dialogue techniques
- Use English punctuation only (no ،)
- Return only valid JSON, no extra text
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You create advanced dialogue flow. Style: {optimal_style}. Return only valid JSON with English punctuation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_json_response(response.choices[0].message.content)

    def add_final_polish(self, topic, information, classification_result, personas_result, enhanced_content_result):
        """
        Main orchestration method: Coordinates all polishing chunks
        """
        print("🎨 Starting enhanced final polishing...")
        print("=" * 50)
        
        try:
            enhanced_content = json.loads(enhanced_content_result)
        except:
            raise ValueError("Invalid enhanced content JSON provided")
        
        # Chunk 1: Polish spontaneous moments
        print("✨ Chunk 1: Polishing spontaneous moments...")
        try:
            current_spontaneous = enhanced_content.get("spontaneous_moments", {})
            polished_spontaneous_json = self.polish_spontaneous_moments(
                topic, classification_result, personas_result, current_spontaneous
            )
            polished_spontaneous = json.loads(polished_spontaneous_json)
            enhanced_content["spontaneous_moments"] = polished_spontaneous
            print("✅ Spontaneous moments polished successfully")
        except Exception as e:
            print(f"⚠️ Error polishing spontaneous moments: {e}")
        
        time.sleep(1)
        
        # Chunk 2: Polish personality interactions
        print("✨ Chunk 2: Polishing personality interactions...")
        try:
            current_personality = enhanced_content.get("personality_interactions", {})
            polished_personality_json = self.polish_personality_interactions(
                topic, classification_result, personas_result, current_personality
            )
            polished_personality = json.loads(polished_personality_json)
            enhanced_content["personality_interactions"] = polished_personality
            print("✅ Personality interactions polished successfully")
        except Exception as e:
            print(f"⚠️ Error polishing personality interactions: {e}")
        
        time.sleep(1)
        
        # Chunk 3: Create shared experiences
        print("✨ Chunk 3: Creating shared experiences...")
        try:
            shared_experiences_json = self.create_shared_experiences(
                topic, classification_result, personas_result
            )
            shared_experiences = json.loads(shared_experiences_json)
            enhanced_content["shared_experiences"] = shared_experiences
            print("✅ Shared experiences created successfully")
        except Exception as e:
            print(f"⚠️ Error creating shared experiences: {e}")
            enhanced_content["shared_experiences"] = self._get_default_shared_experiences()
        
        time.sleep(1)
        
        # Chunk 4: Create contemporary relevance
        print("✨ Chunk 4: Creating contemporary relevance...")
        try:
            contemporary_relevance_json = self.create_contemporary_relevance(
                topic, classification_result, personas_result
            )
            contemporary_relevance = json.loads(contemporary_relevance_json)
            enhanced_content["contemporary_relevance"] = contemporary_relevance
            print("✅ Contemporary relevance created successfully")
        except Exception as e:
            print(f"⚠️ Error creating contemporary relevance: {e}")
            enhanced_content["contemporary_relevance"] = self._get_default_contemporary_relevance()
        
        time.sleep(1)
        
        # Chunk 5: Enhance cultural context
        print("✨ Chunk 5: Enhancing cultural context...")
        try:
            current_cultural = enhanced_content.get("cultural_context", {})
            enhanced_cultural_json = self.enhance_cultural_context(
                topic, classification_result, current_cultural
            )
            enhanced_cultural = json.loads(enhanced_cultural_json)
            enhanced_content["cultural_context"] = enhanced_cultural
            print("✅ Cultural context enhanced successfully")
        except Exception as e:
            print(f"⚠️ Error enhancing cultural context: {e}")
        
        time.sleep(1)
        
        # Chunk 6: Create advanced dialogue flow
        print("✨ Chunk 6: Creating advanced dialogue flow...")
        try:
            advanced_flow_json = self.create_advanced_dialogue_flow(
                topic, classification_result, personas_result
            )
            advanced_flow = json.loads(advanced_flow_json)
            enhanced_content["advanced_dialogue_flow"] = advanced_flow
            print("✅ Advanced dialogue flow created successfully")
        except Exception as e:
            print(f"⚠️ Error creating advanced dialogue flow: {e}")
            enhanced_content["advanced_dialogue_flow"] = self._get_default_dialogue_flow()
        
        print("=" * 50)
        print("🎉 Enhanced final polishing completed!")
        
        return json.dumps(enhanced_content, ensure_ascii=False, indent=2)

    def _clean_json_response(self, response):
        """Enhanced JSON cleaning method"""
        response = response.strip()
        
        # Remove any text before first { and after last }
        start_idx = response.find('{')
        end_idx = response.rfind('}')
        
        if start_idx != -1 and end_idx != -1:
            clean_json = response[start_idx:end_idx+1]
        else:
            clean_json = response
        
        # Replace Arabic punctuation with English equivalents
        clean_json = clean_json.replace('،', ',')
        clean_json = clean_json.replace('"', '"')
        clean_json = clean_json.replace('"', '"')
        clean_json = clean_json.replace(''', "'")
        clean_json = clean_json.replace(''', "'")
        
        # Fix common JSON issues
        import re
        clean_json = re.sub(r',(\s*[}\]])', r'\1', clean_json)
        
        return clean_json

    def _get_default_shared_experiences(self):
        """Default shared experiences"""
        return {
            "common_ground": [
                "اهتمام مشترك بتطوير المجتمع العربي",
                "فهم عميق لأهمية الحفاظ على الهوية الثقافية",
                "خبرة في التعامل مع التحديات المعاصرة"
            ],
            "generational_perspectives": [
                "اختلاف في تجربة التكنولوجيا بين الأجيال",
                "وجهات نظر متنوعة حول التغيير الاجتماعي"
            ],
            "professional_overlaps": [
                "اهتمام مشترك بالتعليم والتطوير",
                "فهم لأهمية التواصل الفعال"
            ],
            "cultural_touchstones": [
                "تقدير للتراث العربي الأصيل",
                "اهتمام بمستقبل الثقافة العربية",
                "فهم لقيم الضيافة والكرم العربي"
            ]
        }

    def _get_default_contemporary_relevance(self):
        """Default contemporary relevance"""
        return {
            "current_events": [
                "التطورات التقنية الحديثة في العالم العربي",
                "المبادرات الثقافية الجديدة في المنطقة"
            ],
            "future_implications": [
                "أهمية الاستثمار في الشباب العربي",
                "ضرورة مواكبة التطورات العالمية"
            ],
            "regional_perspectives": [
                "تنوع الآراء بين البلدان العربية",
                "الحلول المحلية للتحديات العامة"
            ],
            "global_connections": [
                "موقع العالم العربي في الساحة الدولية",
                "التفاعل مع الثقافات العالمية"
            ]
        }

    def _get_default_dialogue_flow(self):
        """Default advanced dialogue flow"""
        return {
            "conversation_rhythms": [
                "التنويع بين الأسئلة العميقة والخفيفة",
                "إعطاء مساحة للتفكير والتأمل"
            ],
            "bridge_phrases": [
                "هذا يقودنا إلى نقطة مهمة",
                "بناءً على ما ذكرت"
            ],
            "emphasis_techniques": [
                "التكرار الذكي للنقاط المهمة",
                "استخدام القصص للتوضيح"
            ],
            "recovery_strategies": [
                "العودة بلطف إلى الموضوع الأساسي",
                "تحويل الخلاف إلى نقاش بناء"
            ],
            "engagement_boosters": [
                "طرح أسئلة تفاعلية",
                "ربط الموضوع بتجارب المستمعين"
            ]
        }

    def validate_final_outline(self, final_json):
        """Enhanced validation for the polished outline"""
        try:
            final_outline = json.loads(final_json)
            missing_elements = []
            
            # Check new sections
            required_new_sections = [
                "shared_experiences", "contemporary_relevance", "advanced_dialogue_flow"
            ]
            for section in required_new_sections:
                if section not in final_outline:
                    missing_elements.append(section)
            
            # Check enhanced sections have more content
            spont_moments = final_outline.get("spontaneous_moments", {})
            required_spont_fields = ["natural_interruptions", "emotional_reactions", "personal_stories", "humorous_moments"]
            for field in required_spont_fields:
                if field not in spont_moments:
                    missing_elements.append(f"spontaneous_moments.{field}")
                elif len(spont_moments.get(field, [])) < 3:
                    missing_elements.append(f"spontaneous_moments.{field} (needs at least 3 items)")
            
            if missing_elements:
                return False, f"Missing enhanced elements: {missing_elements}"
            
            return True, "Enhanced final outline validation successful - ready for premium script generation"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

# Usage:
# polisher = EnhancedFinalPolishEnhancer(deployment, "Fanar-C-1-8.7B")
# final_outline = polisher.add_final_polish(topic, information, classification_result, personas_result, enhanced_content_result)

In [ ]:
polisher = EnhancedFinalPolishEnhancer(deployment, "Fanar-C-1-8.7B")
final_outline = polisher.add_final_polish(topic, information, classification_result, persona_result, enhanced_result)

🎨 Starting enhanced final polishing...
✨ Chunk 1: Polishing spontaneous moments...
⚠️ Error polishing spontaneous moments: Expecting ',' delimiter: line 3 column 115 (char 145)
✨ Chunk 2: Polishing personality interactions...
⚠️ Error polishing personality interactions: Invalid control character at: line 2 column 376 (char 377)
✨ Chunk 3: Creating shared experiences...
✅ Shared experiences created successfully
✨ Chunk 4: Creating contemporary relevance...
✅ Contemporary relevance created successfully
✨ Chunk 5: Enhancing cultural context...
⚠️ Error enhancing cultural context: Expecting value: line 19 column 5 (char 1091)
✨ Chunk 6: Creating advanced dialogue flow...
✅ Advanced dialogue flow created successfully
🎉 Enhanced final polishing completed!


In [29]:
print(final_outline)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "لمى عبد الله",
      "background": "صحفية متخصصة في الشأن التكنولوجي في إحدى الجرائد المحلية",
      "speaking_style": "تطرح أسئلة مفتوحة لتعميق النقاش وتعزز الحديث بموضوعية واضحة"
    },
    "guest": {
      "name": "علي محسن",
      "background": "باحث في علوم الكمبيوتر وأستاذ مساعد في جامعة حكومية",
      "speaking_style": "يعرض أفكارًا فنية بأسلوب سلس ويتفاعل مع المواضيع بتحليل عميق"
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحبا أصدقائي,",
      "podcast_introduction": "انضموا الآن إلى حلقتنا حول الذكاء الصناعي وتراثنا العربي الغني.",
      "episode_hook": "كيف نوازن بين الابتكار والتقاليد?",
      "tone_guidance": "تحليلي ومتفتح للأفكار",
      "spontaneity_elements": [
        "إن هذا موضوع يهم كل واحد منا بلا شك",
        "لنبدأ بتبادل الأفكار حول هذه المسألة المثيرة"
      ]
    },
    "intro2": {
      "

In [ ]:
# Use the enhanced polisher
polisher = EnhancedFinalPolishEnhancer(deployment, "Fanar-C-1-8.7B")
final_outline = polisher.add_final_polish(
    topic, 
    information, 
    classification_result, 
    persona_result, 
    enhanced_result
)

print("Final polished outline:")
print(final_outline)

# Validate the results
is_valid, message = polisher.validate_final_outline(final_outline)
print(f"Validation: {message}")

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "سامي الجابري",
      "background": "صحفي مهتم بالتكنولوجيا والثقافة العربية.",
      "speaking_style": "يطرح أسئلة مباشرة ويسعى لتوضيح الأفكار بأسلوب بسيط."
    },
    "guest": {
      "name": "د. ليلى العمري",
      "background": "أستاذة جامعية متخصصة في الذكاء الاصطناعي واللغويات.",
      "speaking_style": "تشرح الأفكار بأسلوب أكاديمي مبسط مدعوم بالأمثلة."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحباً بكم مستمعينا في بودكاست 'نبض الثقافة'، حيث نناقش القضايا التي تمس هويتنا وثقافتنا في عالم متغير.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع يشغل بال الكثيرين: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هل يمكن لهذه التقنية أن تصبح حليفاً للثقافة العربية أم أنها تهدد بتهميشها؟",
      "spontaneity_elements": [
 

In [32]:
import json
import time
import re

class EnhancedSectionalScriptGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment
        
        # Enhanced dialogue style examples - clean Arabic only
        self.arabic_dialogue_styles = {
            "حواري": {
                "host_tone": "ودود، فضولي، يطرح أسئلة مفتوحة ويتفاعل بحماس",
                "guest_tone": "خبير، متعاون، يشارك معرفته بوضوح ويتفاعل طبيعياً",
                "interaction_style": "حوار تفاعلي مع تداخل طبيعي وضحك وتعليقات عفوية"
            },
            "تعليمي": {
                "host_tone": "منظم، موضح، يطرح أسئلة تعليمية ويلخص النقاط المهمة",
                "guest_tone": "معلم صبور، يشرح بوضوح ويستخدم أمثلة بسيطة",
                "interaction_style": "حوار منظم مع شرح وتوضيح وأسئلة تفصيلية"
            },
            "تحليلي": {
                "host_tone": "عميق، محلل، يطرح أسئلة معقدة ويناقش التفاصيل",
                "guest_tone": "خبير، يقدم تحليل عميق ويناقش الجوانب المختلفة",
                "interaction_style": "نقاش عميق مع تحليل وتفصيل وربط بين الأفكار"
            },
            "ترفيهي": {
                "host_tone": "مرح، خفيف، يضيف لمسات فكاهية ويحافظ على الأجواء الممتعة",
                "guest_tone": "مسترخي، طريف، يشارك قصص ممتعة ويتفاعل بمرح",
                "interaction_style": "حوار مرح مع ضحك وقصص طريفة وتعليقات خفيفة"
            }
        }
        
        # Natural Arabic fillers and transitions
        self.arabic_fillers = {
            "thinking": ["اممم", "يعني", "شوف", "خلاص", "طبعاً"],
            "agreement": ["بالضبط", "صحيح", "أكيد", "تماماً", "فعلاً"],
            "hesitation": ["يعني كيف أقول", "اه ما أدري", "مش عارف كيف أوضح"],
            "excitement": ["واو", "يا الله", "ما شاء الله", "الله يعطيك العافية"],
            "connecting": ["بس", "لكن", "وبعدين", "يا أخي", "اسمع"],
            "gulf_light": ["شلون", "وش رايك", "زين", "ماشي الحال", "الله يعافيك"]
        }

    def generate_intro_dialogue(self, topic, final_outline_result, optimal_style):
        """
        Chunk 1: Generate natural intro dialogue (intro1 + intro2)
        """
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract key elements
        conv_flow = outline.get("conversation_flow", {})
        intro1 = conv_flow.get("intro1", {})
        intro2 = conv_flow.get("intro2", {})
        personas = outline.get("personas", {})
        spontaneous_moments = outline.get("spontaneous_moments", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        host_bg = personas.get("host", {}).get("background", "")
        guest_bg = personas.get("guest", {}).get("background", "")
        
        # Get style guidance
        style_guide = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])

        prompt = f"""
You are an expert Arabic dialogue writer for natural podcast conversations.

Task: Write a natural, engaging introduction dialogue between host and guest.

CONTEXT:
Topic: {topic}
Style: {optimal_style}
Host: {host_name} - {host_bg}
Guest: {guest_name} - {guest_bg}

Style Guidance:
- Host tone: {style_guide["host_tone"]}
- Guest tone: {style_guide["guest_tone"]}
- Interaction style: {style_guide["interaction_style"]}

OUTLINE ELEMENTS TO INCORPORATE:
Intro1 Elements:
- Opening: {intro1.get('opening_line', '')}
- Podcast intro: {intro1.get('podcast_introduction', '')}
- Hook: {intro1.get('episode_hook', '')}
- Spontaneous elements: {intro1.get('spontaneity_elements', [])}

Intro2 Elements:
- Topic introduction: {intro2.get('topic_introduction', '')}
- Guest welcome: {intro2.get('guest_welcome', '')}
- Guest bio: {intro2.get('guest_bio_highlight', '')}
- Cultural connections: {intro2.get('cultural_connections', [])}

Available spontaneous elements: {spontaneous_moments.get('natural_interruptions', [])[:2]}
Cultural references: {cultural_context.get('proverbs_sayings', [])[:2]}

REQUIREMENTS:
1. Start with {host_name} speaking alone (welcome and topic intro)
2. Naturally bring {guest_name} into the conversation
3. Use 75% Modern Standard Arabic, 25% light Gulf dialect touches
4. Include natural fillers like: اممم، يعني، طبعاً، شوف، بالضبط
5. Add natural pauses and interactions
6. Make conversation flow authentically
7. Duration: 2-3 minutes of natural dialogue
8. End with smooth transition to main discussion

CRITICAL LANGUAGE REQUIREMENTS:
- Write ONLY in Arabic (Modern Standard + light Gulf touches)
- NO English words except technical terms if absolutely necessary
- NO Chinese characters or symbols
- NO emojis or special formatting
- NO code-like text or technical annotations
- Use natural Arabic conversation patterns

FORMAT:
{host_name}: [dialogue text]
{guest_name}: [dialogue text]
[pause indicators like: فترة صمت قصيرة، يضحك، يتردد]

Write the complete introduction dialogue:
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert Arabic dialogue writer. Write only in Arabic. No English, Chinese, or emojis. Focus on natural conversation flow in {optimal_style} style."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return self._clean_arabic_dialogue(response.choices[0].message.content)

    def generate_discussion_segment(self, topic, final_outline_result, optimal_style, intro_dialogue, segment_points):
        """
        Chunk 2: Generate discussion segment covering specific points
        
        Args:
            segment_points: List of discussion points to cover in this segment
        """
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract key elements
        personas = outline.get("personas", {})
        spontaneous_moments = outline.get("spontaneous_moments", {})
        personality_interactions = outline.get("personality_interactions", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style guidance
        style_guide = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        # Build points summary for this segment
        points_text = ""
        for i, point in enumerate(segment_points):
            points_text += f"""
النقطة {i+1}: {point.get('point_title', '')}
- الزاوية الشخصية: {point.get('personal_angle', '')}
- المحفزات التلقائية: {point.get('spontaneous_triggers', [])}
- نقاط الخلاف المحتملة: {point.get('disagreement_points', '')}
- المراجع الثقافية: {point.get('cultural_references', [])}
- المحفزات العاطفية: {point.get('emotional_triggers', '')}
"""

        prompt = f"""
You are an expert Arabic dialogue writer for natural podcast discussions.

Task: Write natural discussion dialogue covering the specified points.

CONTEXT:
Topic: {topic}
Style: {optimal_style}
Host: {host_name} - Expert interviewer
Guest: {guest_name} - Topic expert

Style Guidance:
- Host approach: {style_guide["host_tone"]}
- Guest approach: {style_guide["guest_tone"]}
- Interaction style: {style_guide["interaction_style"]}

PREVIOUS CONTEXT (end of intro):
{intro_dialogue[-200:]}

DISCUSSION POINTS TO COVER:
{points_text}

PERSONALITY DYNAMICS:
- Natural chemistry: {personality_interactions.get('natural_chemistry', '')}
- Potential tensions: {personality_interactions.get('tension_points', '')}
- Collaboration style: {personality_interactions.get('collaboration_moments', '')}

SPONTANEOUS ELEMENTS TO USE:
- Natural interruptions: {spontaneous_moments.get('natural_interruptions', [])}
- Emotional reactions: {spontaneous_moments.get('emotional_reactions', [])}
- Personal stories: {spontaneous_moments.get('personal_stories', [])}
- Light humor: {spontaneous_moments.get('humorous_moments', [])}

CULTURAL ELEMENTS:
- Proverbs/sayings: {cultural_context.get('proverbs_sayings', [])}
- Regional references: {cultural_context.get('regional_references', [])}

REQUIREMENTS:
1. Start with smooth transition from previous section
2. Cover all discussion points naturally (don't follow rigid order)
3. Include natural disagreements and agreements
4. Use spontaneous elements authentically
5. Integrate cultural references naturally
6. Show character personalities through dialogue
7. Include natural interruptions and overlaps
8. Duration: 3-4 minutes of flowing discussion
9. End ready for next segment or closing

CRITICAL LANGUAGE REQUIREMENTS:
- Write ONLY in Arabic (Modern Standard + light Gulf dialect)
- NO English, Chinese, or other languages
- NO emojis, symbols, or technical formatting
- Use natural Arabic conversation patterns
- Include authentic hesitations and fillers

FORMAT:
{host_name}: [dialogue text]
{guest_name}: [dialogue text]
[natural pause/reaction indicators in Arabic]

Write the discussion segment:
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You write natural Arabic dialogue. Only Arabic language. No English/Chinese/emojis. Focus on authentic {optimal_style} conversation flow."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )

        return self._clean_arabic_dialogue(response.choices[0].message.content)

    def generate_closing_dialogue(self, topic, final_outline_result, optimal_style, previous_dialogue):
        """
        Chunk 3: Generate natural closing dialogue
        """
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract key elements
        conv_flow = outline.get("conversation_flow", {})
        closing = conv_flow.get("closing", {})
        personas = outline.get("personas", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style guidance
        style_guide = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])

        prompt = f"""
You are an expert Arabic dialogue writer for natural podcast closings.

Task: Write a natural, impactful closing dialogue.

CONTEXT:
Topic: {topic}
Style: {optimal_style}
Host: {host_name}
Guest: {guest_name}

Style Guidance:
- Host approach: {style_guide["host_tone"]}
- Guest approach: {style_guide["guest_tone"]}
- Closing style: {style_guide["interaction_style"]}

PREVIOUS CONTEXT (end of discussion):
{previous_dialogue[-300:]}

CLOSING ELEMENTS FROM OUTLINE:
Conclusion:
- Main takeaways: {closing.get('conclusion', {}).get('main_takeaways', '')}
- Guest final message: {closing.get('conclusion', {}).get('guest_final_message', '')}
- Host closing thoughts: {closing.get('conclusion', {}).get('host_closing_thoughts', '')}

Outro:
- Guest appreciation: {closing.get('outro', {}).get('guest_appreciation', '')}
- Audience thanks: {closing.get('outro', {}).get('audience_thanks', '')}
- Call to action: {closing.get('outro', {}).get('call_to_action', '')}
- Final goodbye: {closing.get('outro', {}).get('final_goodbye', '')}

CULTURAL ELEMENTS:
- Proverbs for closing: {cultural_context.get('proverbs_sayings', [])}
- Cultural wisdom: {cultural_context.get('regional_references', [])}

REQUIREMENTS:
1. Start with natural transition from discussion
2. Summarize key insights naturally (not mechanically)
3. Let {guest_name} share meaningful final thoughts
4. {host_name} adds personal closing reflections
5. Express genuine appreciation for guest and audience
6. Include natural call for engagement
7. Use cultural element for memorable ending
8. Leave positive, inspiring final impression
9. Duration: 1-2 minutes of impactful closing

CRITICAL LANGUAGE REQUIREMENTS:
- Write ONLY in Arabic (Modern Standard + light Gulf dialect)
- NO English, Chinese, or other languages
- NO emojis or special formatting
- Use warm, authentic Arabic expressions
- Natural closing conversation patterns

FORMAT:
{host_name}: [dialogue text]
{guest_name}: [dialogue text]
[natural indicators in Arabic like: يبتسم، يشكر، فترة تأمل]

Write the complete closing dialogue:
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You write natural Arabic closing dialogue. Only Arabic language. No English/Chinese/emojis. Create warm, impactful {optimal_style} ending."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )

        return self._clean_arabic_dialogue(response.choices[0].message.content)

    def generate_complete_script(self, topic, final_outline_result):
        """
        Main orchestration method: Generate complete script using sectional approach
        """
        print("🎙️ بدء توليد السكريبت المطور بالطريقة التقسيمية...")
        print("=" * 50)
        
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Determine optimal style
        optimal_style = self._determine_optimal_style(outline)
        
        print(f"📋 الأسلوب المحدد: {optimal_style}")
        
        # Chunk 1: Generate intro dialogue
        print("📝 الجزء الأول: توليد حوار المقدمة...")
        try:
            intro_dialogue = self.generate_intro_dialogue(topic, final_outline_result, optimal_style)
            print("✅ تم إنجاز حوار المقدمة بنجاح")
        except Exception as e:
            print(f"⚠️ خطأ في توليد المقدمة: {e}")
            intro_dialogue = self._get_fallback_intro(topic, outline, optimal_style)
            print("📝 تم استخدام مقدمة احتياطية")
        
        time.sleep(1)
        
        # Chunk 2: Generate discussion segments
        print("📝 الجزء الثاني: توليد حوار النقاش الرئيسي...")
        discussion_dialogue = ""
        
        # Split discussion points into manageable segments
        main_discussion = outline.get("conversation_flow", {}).get("main_discussion", [])
        segments = self._split_discussion_points(main_discussion)
        
        for i, segment_points in enumerate(segments):
            print(f"  توليد مقطع النقاش {i+1}/{len(segments)}...")
            try:
                segment_dialogue = self.generate_discussion_segment(
                    topic, final_outline_result, optimal_style, 
                    intro_dialogue if i == 0 else discussion_dialogue, 
                    segment_points
                )
                discussion_dialogue += segment_dialogue + "\n\n"
                print(f"  ✅ تم إنجاز مقطع النقاش {i+1}")
            except Exception as e:
                print(f"  ⚠️ خطأ في مقطع {i+1}: {e}")
                fallback_segment = self._get_fallback_discussion_segment(segment_points, optimal_style)
                discussion_dialogue += fallback_segment + "\n\n"
                print(f"  📝 تم استخدام مقطع احتياطي {i+1}")
            
            time.sleep(0.5)
        
        print("✅ تم إنجاز حوار النقاش الرئيسي")
        time.sleep(1)
        
        # Chunk 3: Generate closing dialogue
        print("📝 الجزء الثالث: توليد حوار الختام...")
        try:
            closing_dialogue = self.generate_closing_dialogue(
                topic, final_outline_result, optimal_style, discussion_dialogue
            )
            print("✅ تم إنجاز حوار الختام بنجاح")
        except Exception as e:
            print(f"⚠️ خطأ في توليد الختام: {e}")
            closing_dialogue = self._get_fallback_closing(topic, outline, optimal_style)
            print("📝 تم استخدام ختام احتياطي")
        
        # Combine all parts
        complete_script = f"""=== مقدمة البودكاست ===
{intro_dialogue}

=== النقاش الرئيسي ===
{discussion_dialogue.strip()}

=== ختام البودكاست ===
{closing_dialogue}"""
        
        print("=" * 50)
        print("🎉 تم إنجاز السكريبت الكامل بنجاح!")
        
        return {
            "intro": intro_dialogue,
            "main_discussion": discussion_dialogue.strip(),
            "closing": closing_dialogue,
            "complete_script": complete_script,
            "script_length": len(complete_script),
            "estimated_duration": "10-12 دقيقة تقريباً",
            "style_used": optimal_style,
            "quality_score": self._assess_script_quality(complete_script)
        }

    def _clean_arabic_dialogue(self, dialogue_text):
        """Clean the generated dialogue to ensure quality"""
        # Remove any non-Arabic characters except basic punctuation
        cleaned = re.sub(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s\[\]\(\):\-\.،؟!«»]+', '', dialogue_text)
        
        # Remove emojis and special symbols
        cleaned = re.sub(r'[😀-🙏🚀-🛿⚡-⚿✨-❿💀-💯🔀-🔿]', '', cleaned)
        
        # Clean up extra spaces and line breaks
        cleaned = re.sub(r'\n\s*\n\s*\n+', '\n\n', cleaned)
        cleaned = re.sub(r'[ \t]+', ' ', cleaned)
        
        # Ensure proper Arabic punctuation
        cleaned = cleaned.replace('?', '؟').replace(';', '؛')
        
        return cleaned.strip()

    def _determine_optimal_style(self, outline):
        """Determine the optimal dialogue style from the outline"""
        language_style = outline.get("language_style", {})
        formality = language_style.get("formality_level", "")
        
        if "حواري" in formality or "ودي" in formality:
            return "حواري"
        elif "تحليلي" in formality:
            return "تحليلي"
        elif "مرح" in formality or "ترفيهي" in formality:
            return "ترفيهي"
        else:
            return "تعليمي"

    def _split_discussion_points(self, main_discussion):
        """Split discussion points into manageable segments"""
        if len(main_discussion) <= 2:
            return [main_discussion]
        elif len(main_discussion) == 3:
            return [main_discussion[:2], [main_discussion[2]]]
        else:
            # For more points, split into roughly equal segments
            mid = len(main_discussion) // 2
            return [main_discussion[:mid], main_discussion[mid:]]

    def _get_fallback_intro(self, topic, outline, style):
        """Generate fallback intro if main generation fails"""
        personas = outline.get("personas", {})
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        return f"""{host_name}: مرحباً بكم مستمعينا الكرام، في حلقة جديدة نناقش فيها موضوع {topic}. معي اليوم ضيف متميز لمناقشة هذا الموضوع المهم.

{guest_name}: أهلاً وسهلاً، شكراً على الاستضافة. سعيد بوجودي معكم لمناقشة هذا الموضوع الشيق.

{host_name}: ممتاز، دعنا نبدأ بالتعمق في تفاصيل هذا الموضوع المثير للاهتمام."""

    def _get_fallback_discussion_segment(self, points, style):
        """Generate fallback discussion segment"""
        return """المقدم: هذا موضوع معقد فعلاً، ما رأيك في التحديات الرئيسية؟

الضيف: بصراحة، هناك عدة جوانب مهمة يجب أن نفكر فيها بعناية.

المقدم: ممكن توضح أكثر؟

الضيف: طبعاً، أعتقد أن الأمر يحتاج دراسة عميقة للوصول لحلول فعالة."""

    def _get_fallback_closing(self, topic, outline, style):
        """Generate fallback closing if main generation fails"""
        personas = outline.get("personas", {})
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        return f"""{host_name}: في ختام حلقتنا اليوم، أشكرك {guest_name} على هذا النقاش المفيد حول {topic}.

{guest_name}: شكراً لك على الاستضافة، كان نقاش ممتع ومفيد.

{host_name}: وشكراً لكم مستمعينا الكرام على متابعتكم. نلقاكم في حلقة قادمة بإذن الله."""

    def _assess_script_quality(self, script):
        """Assess the quality of the generated script"""
        # Simple quality metrics
        arabic_ratio = len(re.findall(r'[\u0600-\u06FF]', script)) / len(script) if script else 0
        dialogue_balance = script.count('المقدم:') + script.count('الضيف:')
        
        quality_score = min(100, int(arabic_ratio * 70 + min(dialogue_balance * 5, 30)))
        return quality_score

    def validate_script_quality(self, script_result):
        """Validate the quality of the generated script"""
        try:
            complete_script = script_result.get("complete_script", "")
            
            validation = {
                "has_intro": "=== مقدمة البودكاست ===" in complete_script,
                "has_discussion": "=== النقاش الرئيسي ===" in complete_script,
                "has_closing": "=== ختام البودكاست ===" in complete_script,
                "arabic_content": bool(re.search(r'[\u0600-\u06FF]', complete_script)),
                "no_foreign_text": not bool(re.search(r'[a-zA-Z\u4e00-\u9fff]', complete_script)),
                "balanced_dialogue": complete_script.count('المقدم:') >= 3 and complete_script.count('الضيف:') >= 3,
                "quality_score": script_result.get("quality_score", 0)
            }
            
            validation["overall_valid"] = all([
                validation["has_intro"],
                validation["has_discussion"], 
                validation["has_closing"],
                validation["arabic_content"],
                validation["no_foreign_text"]
            ])
            
            return validation
        except:
            return {"overall_valid": False, "error": "Validation failed"}

# Usage:
# generator = EnhancedSectionalScriptGenerator(deployment, "Fanar-C-1-8.7B")
# script_result = generator.generate_complete_script(topic, final_outline_result)

In [34]:
# Generate high-quality sectional script
generator = EnhancedSectionalScriptGenerator(deployment, "Fanar-C-1-8.7B")
script_result = generator.generate_complete_script(topic, final_outline)

print("Generated Script:")
print(script_result["complete_script"])

# Validate script quality
validation = generator.validate_script_quality(script_result)
print(f"Script Quality: {validation['quality_score']}/100")
print(f"Validation Status: {'✅ Passed' if validation['overall_valid'] else '❌ Failed'}")

# Check individual components
print(f"Intro: {len(script_result['intro'])} characters")
print(f"Discussion: {len(script_result['main_discussion'])} characters") 
print(f"Closing: {len(script_result['closing'])} characters")
print(f"Estimated Duration: {script_result['estimated_duration']}")

🎙️ بدء توليد السكريبت المطور بالطريقة التقسيمية...
📋 الأسلوب المحدد: تحليلي
📝 الجزء الأول: توليد حوار المقدمة...
✅ تم إنجاز حوار المقدمة بنجاح
📝 الجزء الثاني: توليد حوار النقاش الرئيسي...
  توليد مقطع النقاش 1/2...
  ✅ تم إنجاز مقطع النقاش 1
  توليد مقطع النقاش 2/2...
  ✅ تم إنجاز مقطع النقاش 2
✅ تم إنجاز حوار النقاش الرئيسي
📝 الجزء الثالث: توليد حوار الختام...
✅ تم إنجاز حوار الختام بنجاح
🎉 تم إنجاز السكريبت الكامل بنجاح!
Generated Script:
=== مقدمة البودكاست ===
لمى عبد الله: مرحباً أصدقاء! نحنُ اليومِ في رحلةٍ شيقة؛ حيث سنتجوّلْ في عالمِ التقنية ونستكشفُ مدى تاثرياتهِ على هويتِّنَا العرَبِيَّة الفريدَة. فهل تسائَلتم يوماً بينما نُمَدحُ إنجازاتِ المُختبرات العالمية للذكاءِ الاصطناعي، كم تُؤثر تلك النجاحاتعلى شخصيتيَّنَا ومعتقداتيَنَا؟ أمم... إليكم دليلَهذا الاستفسار الدقيق، ينصلهذا الحوار مُناقشةُ ذكيَّةمع الدكتور علي محسِن، الباحث المرموقفي مجالعلومالحاسوبوالأستاذالمساعدةفيجامعة حكومةمحليةمشهورة.مرحباً يا دكتورا، أنتمانحننتظر أفكاركىالقيمةحول هاذه ماهمية قضية !

علي محسن: جازاكِ الل